## Análise de Integração de Serviços para a Recomendação de Produtos

Este *notebook* tem como objetivo analisar o uso conjunto dos modelos criados para fornecer recomendações de produtos. A proposta é receber uma sentença de busca, identificar sua categoria (como se ela fosse um produto), reconhecer sua intenção e gerar recomendações de produtos usando essas informações.

## Bibliotecas e Funções

In [1]:
# General
import sys
import funcy as fp
from pathlib import Path
from typing import List

# Visualization / Presentation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

# Data manipulation and computation
import numpy as np
import pandas as pd

# Embeddings and Similarity
from sklearn.metrics.pairwise import cosine_similarity
import fasttext

# Carregar, além de atualizar frequentemente, código personalizado disponível em ../src
%load_ext autoreload 
%autoreload 2
sys.path.append(str(Path.cwd().parent))
from src import settings
from src.utils.notebooks import display_side_by_side
from src.entities import Product
from src.pipeline.training_pipeline import compute_embeddings_frame
from src.pipeline.inference_pipeline import (load_model_resources,
                                             make_batch_predictions,
                                             make_supervised_intent_classification
                                            )

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)

## Recuperação dos Modelos

Considerando os experimentos feitos nos notebooks [Classificação de Categorias](03.0_Classificacao_de_Categorias.ipynb) e [Classificação de Intenções](04.3_Classificacao_de_Intencoes.ipynb), é possível fazer as tarefas de **classificação de categoria de produto** e de **classificação intenção de busca** usando as funções de inferência já adotadas para a [produtização](06_Consumo_Produtizacao.ipynb). Considerando duas buscas como ponto de partida, pode-se obter a categoria do produto a qual cada uma pertence  e a intenção da busca.

A classificação da busca como um produto é possível sem tratamentos adicionais considerando que o pré-processamento realiza imputação de valores ausentes para as características utilizadas.

In [2]:
queries = ["Roupinha de bebê",
           "Lembrancinha chá de bebê"]

products = [Product(title=query, price=None, concatenated_tags=None)
            for query in queries]

products_with_categories = make_batch_predictions(products)
query_intent = make_supervised_intent_classification(queries)

display_side_by_side([pd.DataFrame(products_with_categories)[['title', 'category']],
                      pd.DataFrame({'Intenções': query_intent})], 
                     ['Classificação de Produtos',
                      'Intenções de Busca'],
                    padding=50)

/media/bruno/f331903b-17cb-447b-b132-e6f1f08f80f8/Development/virtual_envs/teste_HT/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Com o resultado da utilização dos dois classificadores, têm-se a categoria e a intenção da busca. Uma primeira estratégia que pode ser explorada para combinar as informações dos classificadores é a forma de fazer a recuperação de dados para a priorizar as recomendações:
 - **Foco**: Se a pessoa tem uma intenção voltada para um produto específico, é interessante que os produtos exibidos fiquem restritos à categorias próximas. Mostrar uma variedade grande de tipos de produtos pode fazer a pessoa acreditar que os produtos semelhantes ao que ela queria já foram apresentados e não continuar explorando os resultados. Assim, pode-se restringir os resultados à categoria da busca.
 - **Exploração**: Caso a pessoa tenha o objetivo de explorar, é interessante apresentar maior diversificação para que ela possa ver diferentes opções para entender o que quer comprar ou possa refinar sua busca para então torná-la mais direcionada àquilo que entendeu atender sua necessidade. Com isso, é possível recuperar produtos de mais categorias.
 
Esse processo será explorado a seguir.

## Probabilidade de uma Busca ser de uma Categoria

Para conseguir identificar a classe predita para um produto (no caso de foco) ou múltiplas classes de produto que atendam a uma probabilidade mínima (para a intenção de exploração), é preciso personalizar o processo de classificação para se obter as probabilidades em vez da categoria final. O processo é realizado adiante.

In [3]:
CATEGORY_PROBABILITY_THRESHOLD = .25

#query = "Roupinha de bebê"
query = "Lembrancinha chá de bebê"

query_product = Product(title=query, price=None, concatenated_tags=None)

# Identificar a intenção de busca
query_intent = fp.first(make_supervised_intent_classification([query]))

# Recuperar recursos de pré-processamento e classificação
category_preprocessing_model, category_model, category_label_encoder_model = load_model_resources(settings.CATEGORY_CLASSIFICATION_RUN_ID)

# Pré-processar e classificar consulta como um produto
queries_frame = pd.DataFrame([query_product])
preprocessed_products = category_preprocessing_model.predict(queries_frame)
predictions = category_model.predict_proba(preprocessed_products)

# Obter categorias que atendam a um threshold de probabilidade (exploração) ou com a com maior probabilidade (foco)
categories = np.array(category_label_encoder_model.classes_)

if query_intent != 'Foco':
    selected_categories = set(categories[predictions[0] >= CATEGORY_PROBABILITY_THRESHOLD])
else:    
    selected_categories = set([categories[predictions[0].argmax()]])

# Criar dicionário com a probabilidade das categorias selecionadas
category_prob_dict = {category: prob 
                      for prob, category in zip(predictions[0], categories)
                      if category in selected_categories
                     }

display(HTML(f'<strong>Query</strong>: {query}'))
display(HTML(f'<strong>Query Intent</strong>: {query_intent}'))
display(HTML(f'<strong>Selected categories</strong>: {", ".join(selected_categories)}'))

## Pré-processamento para Recuperação

Com a consulta definida e as classes consideradas para a recuperação, de acordo com os critérios definidos para cada intenção, faz-se o carregamento e o pré-processamento dos dados.

In [4]:
columns_to_read = ['product_id', 'title', 'concatenated_tags', 'category', 'view_counts', 'order_counts']

frame = pd.read_csv(Path(settings.DATA_PATH).joinpath('interim', 'training.csv'), usecols=columns_to_read).drop_duplicates('product_id')
display_side_by_side([frame.head()], [f'Dados de Produtos (Registros: {len(frame)})'])

,product_id,title,concatenated_tags,view_counts,order_counts,category
0,11394449,Mandala Espírito Santo,mandala mdf,244,NaN,Decoração
1,15534262,Cartão de Visita,cartao visita panfletos tag adesivos copos long drink canecas,124,NaN,Papel e Cia
2,15877252,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,180,1.0,Bebê
3,15917108,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,34,NaN,Decoração
4,4336889,Álbum de figurinhas dia dos pais,albuns figurinhas pai lucas album fotos,1093,NaN,Lembrancinhas


O pré-processamento envolve:
 - Fazer o filtro dos produtos pelas categorias consideradas;
 - Calcular os *embeddings* relacionados às variávies textuais (*title* e *concatenated tags*)
 - Atribuir a cada categoria de um produto a probabilidade de a busca pertencer a essa categoria.

In [5]:
embeddings_columns = ['title', 'concatenated_tags']
search_frame = (frame
                .loc[lambda f: f['category'].isin(selected_categories)]
                .assign(category_prob=lambda f: f['category'].map(category_prob_dict))
               )
search_frame = pd.concat([search_frame, compute_embeddings_frame(search_frame, embeddings_columns)], axis=1)


display_side_by_side([search_frame.head(2)], [f'Produtos Pré-Processados para Busca (Registros: {len(search_frame)})'])

,product_id,title,concatenated_tags,view_counts,order_counts,category,category_prob,title_embedding,concatenated_tags_embedding
2,15877252,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,180,1.0,Bebê,0.399275,"[-0.009625531, -0.0075558904, 0.04198961, -0.021121407, 0.009023795, -0.057901293, 0.0071319044, 0.0013754551, -0.006352756, -0.032950606, 0.011151789, -0.04205463, 0.011341486, -0.024280775, -0.0039241556, -0.057869595, -0.009882192, 0.0054378184, 0.01935895, -0.056237247, -0.009628549, 0.00954633, 0.004439605, 0.020868437, -0.015975809, -0.02635478, 0.0101419985, -0.035208732, 0.010247768, -0.017244846, 0.005768775, 0.02756766, 0.09242266, -0.016027797, 0.04369178, 0.012324803, 0.019710774, 0.011947667, -0.062149793, 0.023694945, -0.016606329, -0.0005817188, -0.002832982, 0.016965229, 0.0064534354, 0.08155836, -0.004030456, -0.03661938, -0.07426045, 0.08207031, -0.008160974, -0.011370561, 0.011965597, -0.005804021, -0.01498353, -0.019466447, 0.016594771, 0.012576888, -0.08021047, 0.044447504, -0.02228889, -0.019485936, 0.028877527, -0.012301478, 0.04810306, 0.06505157, 0.02173315, 0.0049820887, -0.0028618085, -0.029365782, -0.01686079, 0.03709474, 0.00030628295, -0.048288967, 0.012936994, 0.034661885, -0.002400371, 0.0009890691, 0.018989459, 0.09113573, 0.0637861, -0.011960969, 0.009767355, 0.08708878, -0.026730109, 0.06200397, -0.021442268, 0.00017844886, -0.016166925, -0.016401194, -0.01105361, 0.043394804, -0.0087942025, 0.053579707, 0.08610527, -0.014700484, 0.060181547, 0.031616535, 0.035113093, 0.06032729, ...]","[0.019379606, -0.020988494, -0.0033879082, -0.009727775, 0.03978498, -0.015718888, -0.028393906, -0.019201305, 0.059760742, -0.06380118, 0.029951394, -0.014470777, 0.030149914, -0.004574413, 0.0076197283, -0.025895974, 0.011352579, 0.0047326563, 0.0066905348, -0.032959104, -0.017181316, -0.020785691, 0.008890191, 0.044222914, 0.02613399, 0.009803547, 0.0009075006, 0.0033382091, 0.036685906, -0.01162263, 0.014326158, 0.0004789668, 0.057936084, -0.016628575, 0.02405676, 0.01529185, 0.005546955, 0.048463017, -0.06582837, 0.017569179, -0.030266812, -0.011704776, 0.025409214, 0.068521224, 0.02311313, -0.01649287, 0.017470043, -0.03567913, -0.03193368, 0.034015648, -0.013640204, 0.037855648, -0.018012606, -0.005218176, -0.0039614215, 0.054561906, 0.025894418, 0.04511746, -0.073243305, 0.06593627, -0.010144588, 0.0043264753, 0.004014681, -0.015049703, -0.00058223156, 0.040113464, 0.04875149, 0.047767878, 0.042333975, -0.0011777527, 0.0030900626, 0.002481369, -0.04478432, -0.027379973, 0.022013623, 0.0730473, 0.020801933, 0.0030111112, -0.015459086, 0.042418458, 0.019451004, 0.0051032216, 0.0061770324, 0.087605536, -0.03462994, 0.020290706, 0.039927445, 0.013167984, -0.0068790703, 0.027110748, -0.03137192, 0.05733741, 0.006620887, 0.04312026, 0.053701736, -4.5602523e-05, 0.026222266, 0.025668621, 0.054353327, 0.009070713, ...]"
4,4336889,Álbum de figurinhas dia dos pais,albuns figurinhas pai lucas album fotos,1093,NaN,Lembrancinhas,0.584553,"[-0.027158957, -0.027052542, 0.01778969, -0.028899949, -0.017072415, -0.01605464, -0.0044643283, -0.0142439585, -0.011826115, -0.036974575, 0.014235258, -0.07656851, -0.034678645, 0.007813502, 0.025119407, -0.03965799, 0.046123605, -0.04125409, 0.016830457, -0.040022478, -0.011937154, 0.0074197864, -0.0059881266, -0.043052755, 0.012158862, -0.011359403, 0.0188362, -0.039963964, -0.0038859788, -0.024196401, 0.0076666316, -0.015839504, 0.12284645, 0.00806666, 0.08558771, 0.03521447, 0.018224206, 0.03026221, -0.01079946, 0.009574379, -0.011053006, -0.0032665487, 6.696132e-05, -0.006745437, 0.049170844, 0.09546404, -0.043309, -0.020521523, -0.08508056, 0.044425823, 0.0075528165, 0.008870482, 0.0073616183, -0.023019519, 0.014081991, -0.023127377, 0.036406785, -0.02285758, -0.042841695, -0.0096303895, -0.0061125783, -0.010856534, 0.010407555, 0.0038417503, 0.05738029, 0.069113806, -0.0040099053, -0.017719097, 0.0106506115, 0.045442, -0.0059127975, -0.030889072, 0.

## Recomendação de Produtos

Por fim, são calculados os atributos de similaridade e a computação de relevância para ordenar os resultados:
 - Gerar o *embedding* da busca;
 - Calcular a similaridade entre *embedding* da busca e os *embeddings* do título e das tags concatenadas;
 - Calcular a eficácia de venda de cada produto, considerando *compras*/*visualizações*, limitando o valor a no máximo 1 (caso o número de compras seja maior do que o de visualizações);
 - Calcular a pontuação final do produto considerando a similaridade dos embeddings, probabilidade de a busca pertencer à mesma categoria do produto e o número de compras por visualizações; 
 - Ordenar os produtos pela pontuação final em ordem decrescente e usar a eficácia de venda como critério de desempate caso dois produtos tenham a mesma pontuação.
 
Para cada um dos 3 elementos de similaridade (2 *embeddings* e probabilidade da categoria), além do número de vendas por visualizações, possui um peso atribuído para indicar sua contribuição para a pontuação final. Nesta versão, esses pesos são atribuídos manualmente, mas é possível construir um modelo para identificar qual o melhor peso de cada fator. Um modelo poderia ir além, podendo utilizar os *embeddings* da busca e dos elementos dos produtos para achar relações mais sofisticadas entre os números e que podem ser mais relevantes para determinar a interação com um produto.

In [6]:
def compute_embedding_columns_similarity(base_embeddings_frame: pd.DataFrame,
                                         reference_embedding: np.array,
                                         columns: List[str]) -> pd.DataFrame:
    """Adds, for each column, the similarity between its elements and the reference embedding"""

    similarity_frame = base_embeddings_frame[[]].copy()
    reference_embedding = np.expand_dims(reference_embedding, axis=0)

    for column in columns:
        column_embeddings = np.stack(base_embeddings_frame[f'{column}_embedding'].to_numpy(), axis=0)
        similarity_frame[f'{column}_similarity'] = np.concatenate(cosine_similarity(reference_embedding, column_embeddings), axis=0)

    return similarity_frame


def search_products(base_frame: pd.DataFrame,
                    query: str,
                    items_to_retrieve: int = 10
                   ) -> pd.DataFrame:
    
    TITLE_WEIGHT = 1.0
    TAGS_WEIGHT = 0.75
    CATEGORY_WEIGHT = 0.5
    ORDER_PER_VIEW_WEIGHT = 0.25

    ft_model = fasttext.load_model(str(Path.joinpath(settings.MODELS_PATH, settings.EMBEDDINGS_MODEL)))
    query_embedding = ft_model.get_sentence_vector(query)

    search_frame = base_frame.copy()
    search_frame = pd.concat([search_frame, 
                             compute_embedding_columns_similarity(search_frame, query_embedding, ['title', 'concatenated_tags'])]
                            , axis=1)
    
    columns_to_drop = [item for item in search_frame.columns if item.endswith('_embedding')]    

    return (search_frame
            .fillna({'order_counts': 0,
                     'view_counts': 0})
            .assign(orders_per_views=lambda f: ((f['order_counts']+1)/(f['view_counts']+1)).apply(lambda x: min(1.0, x)))
            .assign(score=lambda f: 
                    (f['title_similarity'] * TITLE_WEIGHT) + 
                    (f['concatenated_tags_similarity'] * TAGS_WEIGHT) + 
                    (f['category_prob'] * CATEGORY_WEIGHT) + 
                    (f['orders_per_views'] * ORDER_PER_VIEW_WEIGHT)
                   )
            .sort_values(by='score', ascending=False)
            .head(items_to_retrieve)
            .drop(columns=columns_to_drop, inplace=False)
           )    
    

display_side_by_side([search_products(search_frame, query, 20)],
                     [f'Resultados para : \'{query}\''])

,product_id,title,concatenated_tags,view_counts,order_counts,category,category_prob,title_similarity,concatenated_tags_similarity,orders_per_views,score
694,11832607,Lembrancinha Chá de Bebê,cha bebe lembrancinhas festa,82,18.0,Lembrancinhas,0.584553,0.964097,0.698957,0.228916,1.837821
15787,10657689,Lembrancinha Chá de Bebê,1 lembrancinhas cha bebe,333,13.0,Lembrancinhas,0.584553,0.964097,0.720370,0.041916,1.807131
15409,5795302,Lembrancinha chá de bebê,lembrancinhas lembrancinha cha bebe,536,3.0,Lembrancinhas,0.584553,0.964097,0.729589,0.007449,1.805428
2510,2297157,Lembrancinha chá de bebê,cha bebe nascimento maternidade latinhas cha bebe bebe lembrancinha,322,0.0,Lembrancinhas,0.584553,0.964097,0.730583,0.003096,1.805085
19107,2986400,Lembrancinha Chá de Bebê,lembrancinhas baby quarto hidratante cha bebe cha bebe menino,581,0.0,Lembrancinhas,0.584553,0.964097,0.715274,0.001718,1.793259
12439,4480359,Lembrancinha Chá de Bebê,cha bernardo lembrancinhas canetas personalizadas cha theo cha bebe lembrancinha cha bebe,509,30.0,Lembrancinhas,0.584553,0.964097,0.688001,0.060784,1.787570
31717,12769565,lembrancinha chá de bebê de elefantinho,cha bebe nascimento lembrancinhas maternidade aniversario latinhas,103,0.0,Lembrancinhas,0.584553,0.927813,0.741722,0.009615,1.778785
32489,9629522,Caneca Personalizada Chá de bebê -Lembrancinha Chá de Bebê,caneca personalizada nascimento lembrancinha maternidade caneca personalizada cha bebe lembrancinha cha bebe caneca personalizada nascimento menina lembrancinha maternidade,337,19.0,Lembrancinhas,0.584553,0.924026,0.724015,0.059172,1.774107
18942,15661963,Latinhas de Lembrancinhas Chá de Bebê,cha menina,38,20.0,Lembrancinhas,0.584553,0.895815,0.601258,0.538462,1.773651
3583,8295646,Lembrancinha de chá de bebê,lembrancinha cha bebe lembrancinha,433,5.0,Lembrancinhas,0.584553,0.939969,0.715692,0.013825,1.772471
